In [1]:
import pandas as pd
import statistics
MRR1 = []
MRR2 = []
pd.options.mode.chained_assignment = None
df = pd.read_csv('C:/Users/lucan/Documents/Uni/Courses/Bachelor Semester Project/BSP1/code/yagoFactsCleaned.csv')
df.columns = ['Subject', 'Predicate', 'Object']

In [2]:
def search(queryList, goldenTriple):
    rpr = 0
    for l in queryList:
        mainIndexList = []
        importance = []
        predicateListFrequencies = []
        for i in range(0, len(queryList)):
            mainIndexList.extend(df.index[df.loc[:, "Subject"] == queryList[i]].tolist())
            mainIndexList.extend(df.index[df.loc[:, "Predicate"] == queryList[i]].tolist())
            mainIndexList.extend(df.index[df.loc[:, "Object"] == queryList[i]].tolist())
        mainIndexListUnique = list(dict.fromkeys(mainIndexList))
        for m in mainIndexListUnique:
            importance.append(100**mainIndexList.count(m))
        df2 = df.iloc[mainIndexListUnique]
        df2.loc[:, "Relevance"] = importance
        predicateList = df2.loc[:, 'Predicate'].tolist()
        predicateListUnique = list(dict.fromkeys(predicateList))
        for k in predicateList:
            predicateListFrequencies.append(predicateList.count(k))
        lowestUniquePredicateFrequency = min(predicateListFrequencies)
        for n in predicateListUnique:
            df2.loc[df.loc[:, 'Predicate'] == n, 'Relevance'] = (df2.loc[df.loc[:, 'Predicate'] == n, 'Relevance']/predicateList.count(n)) * lowestUniquePredicateFrequency
    df2 = df2.sort_values(by=['Relevance'], ascending=False)
    if len(df2) >= 10:
        for p in range(0, len(df2)):
            if df2.iloc[p]["Subject"] == goldenTriple[0] and df2.iloc[p]["Predicate"] == goldenTriple[1] and df2.iloc[p]["Object"] == goldenTriple[2]:
                rpr = 1/(p+1)
                break
    else:
        for p in range(0, 10):
            if df2.iloc[p]["Subject"] == goldenTriple[0] and df2.iloc[p]["Predicate"] == goldenTriple[1] and df2.iloc[p]["Object"] == goldenTriple[2]:
                rpr = 1/(p+1)
                break
    return rpr

In [3]:
%%time
MRR1.append(search(['Rocky_Johnson', 'Dwayne_Johnson'], ['Rocky_Johnson', 'hasChild', 'Dwayne_Johnson']))
MRR1.append(search(['Rocky_Johnson', 'hasChild'], ['Rocky_Johnson', 'hasChild', 'Dwayne_Johnson']))
MRR1.append(search(['Roman_Empire', 'hasCurrency'], ['Roman_Empire', 'hasCurrency', 'Sestertius']))
MRR1.append(search(['Rome', 'http://www.comune.roma.it/'], ['Rome', 'hasWebsite', 'http://www.comune.roma.it/']))
MRR1.append(search(['directed', 'San_Andreas_(film)'], ['Brad_Peyton', 'directed', 'San_Andreas_(film)']))
MRR1.append(search(['wroteMusicFor', 'Cosmopolitan_(film)'], ['Andrew_Lockington', 'wroteMusicFor', 'Cosmopolitan_(film)']))
MRR1.append(search(['Kiribati', 'hasCapital'], ['Kiribati', 'hasCapital', 'South_Tarawa']))
MRR1.append(search(['Charles_the_Fat', 'East_Francia'], ['Charles_the_Fat', 'wasBornIn', 'East_Francia']))
MRR1.append(search(['hasChild', 'Michelle_Obama'], ['Marian_Shields_Robinson', 'hasChild', 'Michelle_Obama']))
MRR1.append(search(['Greenland', 'hasCurrency'], ['Greenland', 'hasCurrency', 'Danish_krone']))
MRR1 = statistics.mean(MRR1)
print(MRR1)

0.35900429088611807
Wall time: 8min 43s


In [4]:
def search(queryList, goldenTriple):
    rpr = 0
    for l in queryList:
        mainIndexList = []
        importance = []
        predicateListFrequencies = []
        for i in range(0, len(queryList)):
            mainIndexList.extend(df.index[df.loc[:, "Subject"] == queryList[i]].tolist())
            mainIndexList.extend(df.index[df.loc[:, "Object"] == queryList[i]].tolist())
        mainIndexListUnique = list(dict.fromkeys(mainIndexList))
        for m in mainIndexListUnique:
            if df.iloc[m]["Predicate"] in queryList:
                importance.append(100**mainIndexList.count(m)*10)
            else:
                importance.append(100**mainIndexList.count(m))
        df2 = df.iloc[mainIndexListUnique]
        df2.loc[:, "Relevance"] = importance
        predicateList = df2.loc[:, 'Predicate'].tolist()
        predicateListUnique = list(dict.fromkeys(predicateList))
        for k in predicateList:
            predicateListFrequencies.append(predicateList.count(k))
        lowestUniquePredicateFrequency = min(predicateListFrequencies)
        for n in predicateListUnique:
            df2.loc[df.loc[:, 'Predicate'] == n, 'Relevance'] = (df2.loc[df.loc[:, 'Predicate'] == n, 'Relevance']/predicateList.count(n)) * lowestUniquePredicateFrequency
    df2 = df2.sort_values(by=['Relevance'], ascending=False)
    if len(df2) >= 10:
        for p in range(0, len(df2)):
            if df2.iloc[p]["Subject"] == goldenTriple[0] and df2.iloc[p]["Predicate"] == goldenTriple[1] and df2.iloc[p]["Object"] == goldenTriple[2]:
                rpr = 1/(p+1)
                break
    else:
        for p in range(0, 10):
            if df2.iloc[p]["Subject"] == goldenTriple[0] and df2.iloc[p]["Predicate"] == goldenTriple[1] and df2.iloc[p]["Object"] == goldenTriple[2]:
                rpr = 1/(p+1)
                break
    return rpr

In [5]:
%%time
MRR2.append(search(['Rocky_Johnson', 'Dwayne_Johnson'], ['Rocky_Johnson', 'hasChild', 'Dwayne_Johnson']))
MRR2.append(search(['Rocky_Johnson', 'hasChild'], ['Rocky_Johnson', 'hasChild', 'Dwayne_Johnson']))
MRR2.append(search(['Roman_Empire', 'hasCurrency'], ['Roman_Empire', 'hasCurrency', 'Sestertius']))
MRR2.append(search(['Rome', 'http://www.comune.roma.it/'], ['Rome', 'hasWebsite', 'http://www.comune.roma.it/']))
MRR2.append(search(['directed', 'San_Andreas_(film)'], ['Brad_Peyton', 'directed', 'San_Andreas_(film)']))
MRR2.append(search(['wroteMusicFor', 'Cosmopolitan_(film)'], ['Andrew_Lockington', 'wroteMusicFor', 'Cosmopolitan_(film)']))
MRR2.append(search(['Kiribati', 'hasCapital'], ['Kiribati', 'hasCapital', 'South_Tarawa']))
MRR2.append(search(['Charles_the_Fat', 'East_Francia'], ['Charles_the_Fat', 'wasBornIn', 'East_Francia']))
MRR2.append(search(['hasChild', 'Michelle_Obama'], ['Marian_Shields_Robinson', 'hasChild', 'Michelle_Obama']))
MRR2.append(search(['Greenland', 'hasCurrency'], ['Greenland', 'hasCurrency', 'Danish_krone']))
MRR2 = statistics.mean(MRR2)
print(MRR2)

0.95
Wall time: 3min 40s
